In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.unit_proccessing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyod.models.ecod import ECOD
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.filter import FILTER

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = UnitDataProcessing(config)

In [ ]:
df_item = features_class.df_item
self = features_class
feature_name = 'f__answer_time_set'
df = df_item[~pd.isnull(df_item[feature_name])].copy()

## Plot the distribution of answer time set

In [ ]:
df[feature_name].hist(bins=48)

## USE ECOD algorithm that makes use of cumulative function and is non-parametric for detecting anomalies in answer time set

In [ ]:
model = ECOD(contamination=0.11)
model.fit(df[[feature_name]])

In [ ]:
df['anomaly'] = model.predict(df[[feature_name]])
df['anomaly'].value_counts(), df['anomaly'].value_counts()/df['anomaly'].count()

In [ ]:
df[df['anomaly']==0][feature_name].min(), df[df['anomaly']==0][feature_name].max()

In [ ]:
# fig = px.box(df, y=feature_name, color='anomaly', points='all')
# fig.show()

In [ ]:
df['total'] =1
df_count = df.groupby(feature_name).agg({'total':'count', 'anomaly':'max'})
df_count = df_count.reset_index()
fig = px.bar(df_count, x=feature_name, y = 'total', color='anomaly')
fig.show()

In [ ]:
df_unit = features_class.df_unit

In [ ]:
df['roster_variable'] = df['roster_level'].astype(str) +df['variable_name'].astype(str) 
data = df.groupby(['interview__id','roster_variable'])['anomaly'].max()
data = data.reset_index()
data = data.groupby('interview__id')['anomaly'].sum()

In [ ]:
df_unit[feature_name.replace('f__','s__')] = df_unit['interview__id'].map(data)
df_unit[feature_name.replace('f__','s__')] = df_unit[feature_name.replace('f__','s__')]/ df_unit['f__number_answered']

In [ ]:
df_unit[feature_name.replace('f__','s__')].hist()

In [ ]:
fig = px.box(df_unit, y=feature_name.replace('f__','s__'))
fig.show()

In [ ]:
df_unit[df_unit[feature_name.replace('f__','s__')]>1]

In [ ]:
df[df['interview__id']=='023ededbb82c4a95bac03dbc18369af1']['anomaly'].sum()

In [ ]:
##########################################################################################################################################################

In [ ]:
df_item['v_r'] = df_item['roster_level'].astype(str) +df_item['variable_name'].astype(str) 
df['v_r'] = df['roster_level'].astype(str) +df['variable_name'].astype(str) 

In [ ]:
df.shape

In [ ]:
x =df.groupby('interview__id').agg({'v_r':'nunique','f__number_answered': 'max','f__number_unanswered': 'max'})

x[x['v_r']==x['f__number_answered']+x['f__number_unanswered']]

In [ ]:
df['006c75cd761b4adf9286e63d9779e679']

In [ ]:
x =df_item.groupby('interview__id').agg({'v_r':'nunique','f__number_answered': 'max','f__number_unanswered': 'max'})
x = x.reset_index()
x[x['v_r']!=x['f__number_answered']+x['f__number_unanswered']]['interview__id'].nunique()

In [ ]:
df[df['f__time_changed']<0]['interview__id'].nunique()

In [ ]:
a = list(df_item[(df_item['type']!='Variable')&(df_item['interview__id']=='14aa0ec823204d258b6fb04162f4fd1a')]['index_col'].unique())
b = list(self.df_active_paradata[(self.df_active_paradata['type']!='Variable')&(self.df_active_paradata['interview__id']=='14aa0ec823204d258b6fb04162f4fd1a')]['index_col'].unique())


In [ ]:
diff = set(a) - set(b)
diff

In [ ]:
df1 = df_item[pd.isnull(df_item['f__answer_time_set'])].copy()
mask = (df1['interview__id']=='14aa0ec823204d258b6fb04162f4fd1a') & (df1['variable_name'] =='toilet')
df1[mask]

In [ ]:
df1 = self.df_paradata.copy()
mask = (df1['interview__id']=='14aa0ec823204d258b6fb04162f4fd1a') #& (df1['variable_name'] =='toilet')
df1[mask]

In [ ]:
df1 = self.df_active_paradata.copy()
mask = (df1['interview__id']=='14aa0ec823204d258b6fb04162f4fd1a') #& (df1['variable_name'] =='toilet')
df1[mask]

In [ ]:
self.df_paradata[self.df_paradata['variable_name']=='GPS']['answers'].astype(str).unique()

In [ ]:
self.df_microdata['value1'] = self.df_microdata['value'].astype(str)
self.df_microdata.groupby('variable_name')['value1'].unique()

In [ ]:
df_item = self._df_microdata[['value', 'type', 'is_integer', 'qnr_seq',
                              'n_answers', 'answer_sequence',
                              'cascade_from_question_id', 'is_filtered_combobox',
                              'index_col'] + self.item_level_columns].copy()

paradata_columns = ['responsible', 'f__answer_year_set', 'f__answer_month_set',
                    'f__answer_day_set', 'f__half_hour', 'f__answer_time_set']
df_item = df_item.merge(self.df_paradata[paradata_columns + ['index_col']], how='left',
                        on='index_col')